<a href="https://colab.research.google.com/github/xRAFPx/AAII/blob/master/RNN_Multivariate_TSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import requests
import json
import csv
import string
import io
import pandas as pd
from tempfile import NamedTemporaryFile
import shutil
from datetime import date
import datetime
import os.path
from os import path
import re
import xlrd
import math
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from getpass import getpass
from functools import reduce
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


# Variáveis

In [ ]:
folder = '/content/drive/My Drive/Colab Notebooks/AII Project'
gitFolder = '/content/AAII/Data'
model_folder = folder + '/Model Output'
bestFileRNN_Multi = model_folder + '/bestResultsRNN_Multi.csv'

#Temperatura
norte_weather_filepath = folder + '/weather_norte.csv'
centro_weather_filepath = folder + '/weather_centro.csv'
lvt_weather_filepath = folder + '/weather_lvt.csv'
algarve_weather_filepath = folder + '/weather_algarve.csv'
alentejo_weather_filepath = folder + '/weather_alentejo.csv'

#Óbitos COVID-19
acores_death_filepath = folder + '/acores_death.csv'
madeira_death_filepath = folder + '/madeira_death.csv'
alentejo_death_filepath = folder + '/alentejo_death.csv'
algarve_death_filepath = folder + '/algarve_death.csv'
lvt_death_filepath = folder + '/lvt_death.csv'
centro_death_filepath = folder + '/centro_death.csv'
norte_death_filepath = folder + '/norte_death.csv'

#Casos Confirmados COVID-19
acores_filepath = folder + '/acores.csv'
madeira_filepath = folder + '/madeira.csv'
alentejo_filepath = folder + '/alentejo.csv'
algarve_filepath = folder + '/algarve.csv'
lvt_filepath = folder + '/lvt.csv'
centro_filepath = folder + '/centro.csv'
norte_filepath = folder + '/norte.csv'

In [ ]:
if path.exists(folder):
  print("No directory error")
else:
  #Create a new directory (a folder) in your Drive
  os.mkdir(folder)
  print("Directory created")

No directory error


In [ ]:
def read_dataset(url):
  df = pd.read_csv(url, parse_dates=[0], index_col=[0], infer_datetime_format=True)
  return df.fillna(value=0)

In [ ]:
df_norte_aux = read_dataset(norte_filepath)
# pd.set_option('display.max_rows', df_norte.shape[0]+1)
# print(df_norte)
df_centro_aux = read_dataset(centro_filepath)
df_lvt_aux = read_dataset(lvt_filepath)
df_alentejo_aux = read_dataset(alentejo_filepath)
df_algarve_aux = read_dataset(algarve_filepath)

In [ ]:
df_norte_aux.shape

(116, 1)

In [ ]:
df_alentejo_death = read_dataset(alentejo_death_filepath)
df_algarve_death = read_dataset(algarve_death_filepath)
df_lvt_death = read_dataset(lvt_death_filepath)
df_centro_death = read_dataset(centro_death_filepath)
df_norte_death = read_dataset(norte_death_filepath)

In [ ]:
df_norte_weather = read_dataset(norte_weather_filepath)
df_centro_weather = read_dataset(centro_weather_filepath)
df_lvt_weather = read_dataset(lvt_weather_filepath)
df_algarve_weather = read_dataset(algarve_weather_filepath)
df_alentejo_weather = read_dataset(alentejo_weather_filepath)

In [ ]:
df_norte_aux[df_norte_aux<0]=0
df_centro_aux[df_centro_aux<0]=0
df_lvt_aux[df_lvt_aux<0]=0
df_alentejo_aux[df_alentejo_aux<0]=0
df_algarve_aux[df_algarve_aux<0]=0

df_norte_death[df_norte_death<0]=0
df_centro_death[df_centro_death<0]=0
df_lvt_death[df_lvt_death<0]=0
df_alentejo_death[df_alentejo_death<0]=0
df_algarve_death[df_algarve_death<0]=0

#Preparação das variáveis

In [ ]:
def create_predictor(df, df_aux):
    df_predictor=df.copy()
    meanMinData = dict(zip(df_aux.index, df_aux['mean_x'].round()))
    meanMaxData = dict(zip(df_aux.index, df_aux['mean_y'].round()))
    df_predictor["temp_max"] = df.index.map(meanMaxData)
    df_predictor["temp_min"] = df.index.map(meanMinData)
    df_predictor = df_predictor.fillna(method='ffill')
    return df_predictor

In [ ]:
def data_normalization(df):
  scaler = MinMaxScaler()
  df = scaler.fit_transform(df)
  return scaler, df

In [ ]:
def build_multiSupervised(X, y, timesteps):
    dataX = X
    dataY = y
    X_s, y_s = list(), list()
    dataset_size = len(dataX)
    for curr_pos in range(dataset_size):
        input_index =  curr_pos + timesteps
        label_index = input_index + 1
        if label_index <= dataset_size:
            X_s.append(dataX[curr_pos:input_index, :])
            y_s.append(dataY[input_index:label_index])
    return np.array(X_s).astype('float32'), np.array(y_s).astype('float32')

In [ ]:
df_norte_predictor_aux = create_predictor(df_norte_death, df_norte_weather)
df_centro_predictor_aux = create_predictor(df_centro_death, df_centro_weather)
df_lvt_predictor_aux = create_predictor(df_lvt_death, df_lvt_weather)
df_alentejo_predictor_aux = create_predictor(df_alentejo_death, df_alentejo_weather)
df_algarve_predictor_aux = create_predictor(df_algarve_death, df_algarve_weather)

In [ ]:
scaler_norte, df_norte_aux = data_normalization(df_norte_aux)
scaler_centro, df_centro_aux = data_normalization(df_centro_aux)
scaler_lvt, df_lvt_aux = data_normalization(df_lvt_aux)
scaler_alentejo, df_alentejo_aux = data_normalization(df_alentejo_aux)
scaler_algarve, df_algarve_aux = data_normalization(df_algarve_aux)

In [ ]:
scaler_norte_predictor, df_norte_predictor_aux = data_normalization(df_norte_predictor_aux)
scaler_centro_predictor, df_centro_predictor_aux = data_normalization(df_centro_predictor_aux)
scaler_lvt_predictor, df_lvt_predictor_aux = data_normalization(df_lvt_predictor_aux)
scaler_alentejo_predictor, df_alentejo_predictor_aux = data_normalization(df_alentejo_predictor_aux)
scaler_algarve_predictor, df_algarve_predictor_aux = data_normalization(df_algarve_predictor_aux)

In [ ]:
timesteps = 4

In [ ]:
df_norte_predictor, df_norte = build_multiSupervised(df_norte_predictor_aux, df_norte_aux, timesteps)

In [ ]:
df_centro_predictor, df_centro = build_multiSupervised(df_centro_predictor_aux, df_centro_aux, timesteps)

In [ ]:
df_algarve_predictor, df_algarve = build_multiSupervised(df_algarve_predictor_aux, df_algarve_aux, timesteps)

In [ ]:
df_alentejo_predictor, df_alentejo = build_multiSupervised(df_alentejo_predictor_aux, df_alentejo_aux, timesteps)

In [ ]:
df_lvt_predictor, df_lvt = build_multiSupervised(df_lvt_predictor_aux, df_lvt_aux, timesteps)

In [ ]:
df_norte_predictor.shape

(112, 4, 3)

In [ ]:
df_norte.shape

(112, 1, 1)

In [ ]:
valid_test_percent = 0.1
df_region_valid_test_size = int(np.round(valid_test_percent * df_norte.shape[0]))
df_region_train_size = df_norte.shape[0] - df_region_valid_test_size*2

In [ ]:
df_region_valid_test_size

11

In [ ]:
df_region_train_size

90

In [ ]:
#TRAIN
X_train_norte = df_norte_predictor[:df_region_train_size]
y_train_norte = df_norte[:df_region_train_size]

X_train_centro = df_centro_predictor[:df_region_train_size]
y_train_centro = df_centro[:df_region_train_size]

X_train_lvt = df_lvt_predictor[:df_region_train_size]
y_train_lvt = df_lvt[:df_region_train_size]

X_train_algarve = df_algarve_predictor[:df_region_train_size]
y_train_algarve = df_algarve[:df_region_train_size]

X_train_alentejo = df_alentejo_predictor[:df_region_train_size]
y_train_alentejo = df_alentejo[:df_region_train_size]


#TEST
X_test_norte = df_norte_predictor[df_region_train_size + df_region_valid_test_size :]
y_test_norte = df_norte[df_region_train_size + df_region_valid_test_size :]

X_test_centro = df_centro_predictor[df_region_train_size + df_region_valid_test_size :]
y_test_centro = df_centro[df_region_train_size + df_region_valid_test_size :]

X_test_lvt = df_lvt_predictor[df_region_train_size + df_region_valid_test_size :]
y_test_lvt = df_lvt[df_region_train_size + df_region_valid_test_size :]

X_test_algarve = df_algarve_predictor[df_region_train_size + df_region_valid_test_size :]
y_test_algarve = df_algarve[df_region_train_size + df_region_valid_test_size :]

X_test_alentejo = df_alentejo_predictor[df_region_train_size + df_region_valid_test_size :]
y_test_alentejo = df_alentejo[df_region_train_size + df_region_valid_test_size :]


#VALIDATION
X_val_norte = df_norte_predictor[df_region_train_size:df_region_train_size + df_region_valid_test_size]
y_val_norte = df_norte[df_region_train_size:df_region_train_size + df_region_valid_test_size]

X_val_centro = df_centro_predictor[df_region_train_size:df_region_train_size + df_region_valid_test_size]
y_val_centro = df_centro[df_region_train_size:df_region_train_size + df_region_valid_test_size]

X_val_lvt = df_lvt_predictor[df_region_train_size:df_region_train_size + df_region_valid_test_size]
y_val_lvt = df_lvt[df_region_train_size:df_region_train_size + df_region_valid_test_size]

X_val_algarve = df_algarve_predictor[df_region_train_size:df_region_train_size + df_region_valid_test_size]
y_val_algarve = df_algarve[df_region_train_size:df_region_train_size + df_region_valid_test_size]

X_val_alentejo = df_alentejo_predictor[df_region_train_size:df_region_train_size + df_region_valid_test_size]
y_val_alentejo = df_alentejo[df_region_train_size:df_region_train_size + df_region_valid_test_size]


#TRAIN+VALIDATION
X_train_val_norte = df_norte_predictor[:df_region_train_size + df_region_valid_test_size]
y_train_val_norte = df_norte[:df_region_train_size + df_region_valid_test_size]

X_train_val_centro = df_centro_predictor[:df_region_train_size + df_region_valid_test_size]
y_train_val_centro = df_centro[:df_region_train_size + df_region_valid_test_size]

X_train_val_lvt = df_lvt_predictor[:df_region_train_size + df_region_valid_test_size]
y_train_val_lvt = df_lvt[:df_region_train_size + df_region_valid_test_size]

X_train_val_algarve = df_algarve_predictor[:df_region_train_size + df_region_valid_test_size]
y_train_val_algarve = df_algarve[:df_region_train_size + df_region_valid_test_size]

X_train_val_alentejo = df_alentejo_predictor[:df_region_train_size + df_region_valid_test_size]
y_train_val_alentejo = df_alentejo[:df_region_train_size + df_region_valid_test_size]

In [ ]:
X_train_norte.shape

(90, 4, 3)

In [ ]:
y_train_norte.shape

(90, 1, 1)

In [ ]:
X_train_val_norte.shape

(101, 4, 3)

In [ ]:
y_test_norte_re = y_test_norte.reshape(y_test_norte.shape[0],y_test_norte.shape[1])
y_train_val_norte_re = y_train_val_norte.reshape(y_train_val_norte.shape[0],y_train_val_norte.shape[1])

In [ ]:
y_test_centro_re = y_test_centro.reshape(y_test_centro.shape[0],y_test_centro.shape[1])
y_train_val_centro_re = y_train_val_centro.reshape(y_train_val_centro.shape[0],y_train_val_centro.shape[1])

In [ ]:
y_test_lvt_re = y_test_lvt.reshape(y_test_lvt.shape[0],y_test_lvt.shape[1])
y_train_val_lvt_re = y_train_val_lvt.reshape(y_train_val_lvt.shape[0],y_train_val_lvt.shape[1])

In [ ]:
y_test_alentejo_re = y_test_alentejo.reshape(y_test_alentejo.shape[0],y_test_alentejo.shape[1])
y_train_val_alentejo_re = y_train_val_alentejo.reshape(y_train_val_alentejo.shape[0],y_train_val_alentejo.shape[1])

In [ ]:
y_test_algarve_re = y_test_algarve.reshape(y_test_algarve.shape[0],y_test_algarve.shape[1])
y_train_val_algarve_re = y_train_val_algarve.reshape(y_train_val_algarve.shape[0],y_train_val_algarve.shape[1])

# RNN

In [ ]:
def rmse(y_true, y_pred):
    #RMSE = tf.sqrt(tf.losses.mean_squared_error(y_pred, y_true))
    #RMSE = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_pred, y_true))))
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

In [ ]:
def save_RNNGridSearch(export_file, best_file, gs_clf, city):
	with open(export_file, 'w') as outfile:
		csvwriter = csv.writer(outfile, delimiter=',')
		#Create the header using the parameter names 
		
		header = ["mean","std"]
		param_names = [param for param in gs_clf.param_distributions.keys()]
		header.extend(param_names)
		csvwriter.writerow(header)
	
		means = gs_clf.cv_results_['mean_test_score']
		stds = gs_clf.cv_results_['std_test_score']
		params = gs_clf.cv_results_['params']

		for mean, stdev, param in zip(means, stds, params):
			row = [mean,stdev]
			# Get the list of parameter settings and add to row
			params = [str(p) for p in param.values()]
			row.extend(params)
			
			csvwriter.writerow(row)
	 
	write_header=1 
	if path.exists(best_file):
		write_header = 0

	with open(best_file, 'a') as bestfile:
		csvwriter = csv.writer(bestfile, delimiter=',')	
		if write_header == 0:
			pass
		else:
			header = ["city","score"]
			param_names = [param for param in gs_clf.param_distributions.keys()]
			header.extend(param_names)
			csvwriter.writerow(header)
	
		mean = gs_clf.best_score_
		params = gs_clf.best_params_

		row = [city, mean]
		# Get the list of parameter settings and add to row
		params = [str(p) for p in params.values()]
		row.extend(params)
		
		csvwriter.writerow(row)

In [ ]:
def RNN_Model(model_type, layers_size, num_layers, dropout_rate, timesteps, features, optimizer, learn_rate):
  model = tf.keras.models.Sequential() 
  for i in range(num_layers):
    if model_type == 'RNN':
      if i == 0 and num_layers > 1:
        model.add(tf.keras.layers.SimpleRNN(int(layers_size), return_sequences=True, input_shape=(timesteps, features)))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))  
      elif i == 0 and num_layers == 1:
        model.add(tf.keras.layers.SimpleRNN(int(layers_size), input_shape=(timesteps, features)))
      elif i == num_layers-1:
        model.add(tf.keras.layers.SimpleRNN(int(layers_size*i)))
      else:
        model.add(tf.keras.layers.SimpleRNN(int(layers_size*i), return_sequences=True))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))  
    elif model_type == 'LSTM':
      if i == 0 and num_layers > 1:
        model.add(tf.keras.layers.LSTM(int(layers_size), return_sequences=True, input_shape=(timesteps, features)))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))  
      elif i == 0 and num_layers == 1:
        model.add(tf.keras.layers.LSTM(int(layers_size), input_shape=(timesteps, features)))
      elif i == num_layers-1:
        model.add(tf.keras.layers.LSTM(int(layers_size*i)))
      else:
        model.add(tf.keras.layers.LSTM(int(layers_size*i), return_sequences=True))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))  
    else:
      if i == 0 and num_layers > 1:
        model.add(tf.keras.layers.GRU(int(layers_size), return_sequences=True, input_shape=(timesteps, features)))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))  
      elif i == 0 and num_layers == 1:
        model.add(tf.keras.layers.GRU(int(layers_size), input_shape=(timesteps, features)))
      elif i == num_layers-1:
        model.add(tf.keras.layers.GRU(int(layers_size*i)))
      else:
        model.add(tf.keras.layers.GRU(int(layers_size*i), return_sequences=True))
        if dropout_rate > 0: model.add(tf.keras.layers.Dropout(dropout_rate))   
  model.add(tf.keras.layers.Dense(layers_size, activation="linear"))  
  model.add(tf.keras.layers.Dense(1, activation="sigmoid"))  
  # model.summary()
  if optimizer == 'sgd_momentum':
    model.compile(optimizer = tf.keras.optimizers.SGD(lr=learn_rate, momentum=0.7), loss = rmse, metrics = ['mae', 'accuracy', rmse])
  elif optimizer == 'adam':
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = rmse, metrics = ['mae', 'accuracy', rmse])
  else:
    model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=learn_rate), loss = rmse, metrics = ['mae', 'accuracy', rmse])
  
  return model


In [ ]:
def RNN_model_search(X, Y, X_test, y_test, scaler, city):

  export_file = model_folder + '/MultiRNN' + city + '.csv'
  #call the model used in the cross validation
  model = KerasClassifier(build_fn=RNN_Model, verbose=1, timesteps = X.shape[1], features = X.shape[2])

  #time series split for nested cross validation
  tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)

  # define the grid search parameters
  opt_pars = {
            "layers_size":[5, 10, 20, 100, 150],
            "num_layers": [1, 2, 3, 4, 5],
            "optimizer": [ "adam", "rmsprop", "sgd_momentum"],
            "learn_rate": [0.01, 0.001, 0.0001],
            "dropout_rate": [0.2, 0.5],
            "epochs": [1000, 10000, 100000],
            "batch_size": [8, 16, 32],
            "model_type": ["RNN", "LSTM", "GRU"]
            }

  grid = RandomizedSearchCV(estimator=model, param_distributions=opt_pars, n_jobs=1, cv=tscv, error_score=0.0, scoring = "neg_mean_squared_error", n_iter=20)
  grid_result = grid.fit(X, Y, verbose=0)

  #Resumo dos resultados
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

  #Guardar os resultados
  save_RNNGridSearch(export_file, best_fileRNN, grid_result, city)

  #Previsão
  prediction = grid_result.predict(X_test)
  print("Prediction:", prediction)
  # scaled_prediction = prediction.reshape(1, -1)
  # scaled_prediction = scaler.inverse_transform(scaled_prediction)
  # scaled_realValue = y_test.reshape(1, -1)
  # scaled_realValue = scaler.inverse_transform(scaled_realValue)
  # for p, r in zip(scaled_prediction, scaled_realValue):
  #   for i in range(len(p)):
  #     print("Prediction:", p[i], "\tReal Value:", r[i])
  score_test = mean_squared_error(y_test, prediction)
  print("Test Score:", score_test)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
grid_search_RNN(X_train_val_norte, y_train_val_norte, X_test_norte, y_test_norte, scaler_norte, "Norte")

In [ ]:
grid_search_RNN(X_train_val_centro, y_train_val_centro, X_test_centro, y_test_centro, scaler_centro, "Centro")

In [ ]:
grid_search_RNN(X_train_val_lvt, y_train_val_lvt, X_test_lvt, y_test_lvt, scaler_lvt, "LVT")

In [ ]:
grid_search_RNN(X_train_val_alentejo, y_train_val_alentejo, X_test_alentejo, y_test_alentejo, scaler_alentejo, "Alentejo")

In [ ]:
grid_search_RNN(X_train_val_algarve, y_train_val_algarve, X_test_algarve, y_test_algarve, scaler_algarve, "Algarve")